In [1]:
import sys
from torchsummary import summary
import functools
import ast
import torch
import random
import numpy as np
import icecream as ic
import time
from IPython.display import clear_output
import matplotlib.pyplot as plt
sys.path.append("../../")
sys.path.append("../")
####### Dataset ############
from data_processor import DataProcessor
######## Search space #########
from search_space.RegNet import RegNet
from search_space.utils import create_widths_plot, scatter_results, get_generation_dfs
######## Training ###############
from trainer import Trainer, TrainerDistillation
from utils.train_cfg import get_cfg, show_cfg
###################################################
random_seed = 1
random.seed(random_seed)
# Set seed for NumPy
np.random.seed(random_seed)
# Set seed for PyTorch
torch.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed)
# Additional steps if using CuDNN (optional, for GPU acceleration)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import os
from datetime import datetime
import itertools
import pandas as pd
import json
from io import StringIO
from coolname import generate_slug
from sklearn.metrics import accuracy_score
import torch.multiprocessing as mp


import torch.nn as nn
import types
import copy
import torch.nn.functional as F
from zcost_proxies.predictive import find_measures

/home/hpc/iwb3/iwb3021h/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
def load_dataset_metadata(dataset_path):
    with open(os.path.join(dataset_path, 'metadata'), "r") as f:
        metadata = json.load(f)
        
    return metadata
# load dataset from file
def load_datasets(data_path, truncate):
    data_path = '../../datasets_val/'+data_path
    train_x = np.load(os.path.join(data_path,'train_x.npy'))
    train_y = np.load(os.path.join(data_path,'train_y.npy'))
    valid_x = np.load(os.path.join(data_path,'valid_x.npy'))
    valid_y = np.load(os.path.join(data_path,'valid_y.npy'))
    test_x = np.load(os.path.join(data_path,'test_x.npy'))
    metadata = load_dataset_metadata(data_path)

    if truncate:
        train_x = train_x[:64]
        train_y = train_y[:64]
        valid_x = valid_x[:64]
        valid_y = valid_y[:64]
        test_x = test_x[:64]

    return (train_x, train_y), \
           (valid_x, valid_y), \
           (test_x), metadata

def validation(model, val_loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    # 4. Calculate Accuracy
    accuracy = correct / total
    print('Accuracy on the test set: {:.2f}%'.format(accuracy * 100))

In [8]:
import json
import os

import kaleido
import pandas as pd
import numpy as np
import plotly.graph_objects as go
######## Search space #########
import sys
sys.path.append("../")
from search_space.RegNet import RegNet
from search_space.utils import create_widths_plot, scatter_results
# Initialize an empty list to store dictionaries
def results_to_df(path, name):
    data = []
    # Open the text file
    with open(path, 'r') as file:
        lines = file.readlines()
        # Initialize an empty dictionary to store data for each block
        block_data = {}
        for line in lines:
            # If the line contains dashes, it indicates the end of a block
            if '-------------------------' in line:
                # If block_data is not empty, add it to the list of data dictionaries
                if block_data:
                    data.append(block_data)
                    # Reset block_data for the next block
                    block_data = {}
            elif 'best_acc' in line:
                continue
            else:
                # Split the line by ':'
                #print(line)
                key, value = line.strip().split(': ')
                # Store the key-value pair in the block_data dictionary
                block_data[key] = value

    # Create a DataFrame from the list of dictionaries
    df = pd.DataFrame(data)

    # Convert columns to appropriate data types if needed
    df['epoch'] = df['epoch'].astype(int)
    df['lr'] = df['lr'].astype(float)
    df['train_acc'] = df['train_acc'].astype(float)
    df['train_loss'] = df['train_loss'].astype(float)
    df['test_acc'] = df['test_acc'].astype(float)
    df['test_acc_top5'] = df['test_acc_top5'].astype(float)
    df['test_loss'] = df['test_loss'].astype(float)
    df['epoch_time'] = df['epoch_time'].astype(float)
    df=df.assign(name=name)
    
    return df

metadata_dict={}
metadata_dict["Caitie"]={'num_classes': 4,
                         'input_shape': [49260, 3, 64, 64],
                         'codename': 'Caitie',
                         'benchmark': 47.008,
                         }
metadata_dict["Adaline"]={'num_classes': 20,
                         'input_shape': [50000, 3, 28, 28],
                         'codename': 'Adaline',
                         'benchmark': 89.85}
metadata_dict["Chester"]={'input_shape': [49998, 12, 8, 8],
                         'codename': 'Chester',
                         'benchmark': 57.826,
                         'num_classes': 3}
metadata_dict["Gutenberg"]={'input_shape': [45000, 1, 27, 18],
                         'codename': 'Gutenberg',
                         'benchmark': 40.98,
                         'num_classes': 6}
metadata_dict["LaMelo"]={'num_classes': 10,
                         'codename': 'LaMelo',
                         'input_shape': [50000, 1, 24, 24],
                         'benchmark': 85.2}
metadata_dict["Mateo"]={'num_classes': 10,
                         'input_shape': [50000, 3, 28, 28],
                         'codename': 'Mateo',
                         'benchmark': 90.87}
metadata_dict["Sadie"]={'input_shape': [50000, 3, 64, 64],
                         'codename': 'Sadie',
                         'benchmark': 80.33,
                         'num_classes': 10}
metadata_dict["Sokoto"]={'input_shape': [50000, 1, 9, 9],
                         'codename': 'Sokoto',
                         'benchmark': 0.0,
                         'num_classes': 9}
metadata_dict["CIFAR10"]={
                            "input_shape": [50000, 3, 32, 32],  
                            "codename": "CIFAR10",             
                            "benchmark": 90.65,                 
                            "num_classes": 10                  
                        }
def get_predictor_data_from_folder(base_folder,studies):

    
    train_data=[]
    for study_name in studies: 
        

        metadata=metadata_dict[study_name]
        rg=RegNet(metadata,
                        W0=[16, 120, 8],
                        WA=[16, 64, 8],
                        WM=[2.05,2.9,0.05],
                        D=[8,22,1], 
                        G=[8,8,8], 
                        base_config=f"../configs/search_space/config.yaml")
        data=[]
        study_folder=f"{base_folder}/{study_name}"
        subjects= os.listdir(study_folder)

        subjects=[subject for subject in subjects if os.path.isdir(f"{study_folder}/{subject}")]
        print(subjects)
        for name in subjects:
            try:
                test_acc=results_to_df(f"{study_folder}/{name}/worklog.txt", f"{name}")[["epoch","test_acc","name"]]
                #test_acc=test_acc.rename(columns={"test_acc":"acc","test_loss":"test_acc"})
                #print(test_acc.columns)
                if len(test_acc)>49:
                    test_acc_piv = test_acc[["epoch","test_acc","name"]].pivot(index='name', columns='epoch', values='test_acc').add_prefix("epoch_").reset_index()
                    test_acc_piv["best_acc"]=test_acc["test_acc"].max()
                    
                    model, info=rg.load_model(config_file=f"{study_folder}/{name}/config.yaml")
                    test_acc_piv["gen"]=1
                    test_acc_piv["num_stages"]=info["num_stages"]
                    test_acc_piv["params"]=info["params"]
                    test_acc_piv["WA"]=info["WA"]
                    test_acc_piv["W0"]=info["W0"]
                    test_acc_piv["WM"]=info["WM"]
                    test_acc_piv["DEPTH"]=info["DEPTH"]
                    test_acc_piv["num_classes"]=metadata["num_classes"]
                    test_acc_piv["num_channels"]=metadata["input_shape"][1]
                    test_acc_piv["benchmark"]=metadata["benchmark"]
                    
                    #print(info)
                    data.append(test_acc_piv)
            except:
                    pass

        data=pd.concat(data)
        train_data.append(data)
    train_data=pd.concat(train_data)
    return train_data
    

In [9]:
from torchvision.transforms import v2
class RandomPixelChange:
    def __init__(self, change_prob=0.1):
        self.change_prob = change_prob
    
    def __call__(self, img):
        # Convert image to numpy array
        img_array = np.array(img).astype(np.float32)
        
        # Normalize the array to [0, 1]
        img_array = (img_array - img_array.min()) / (img_array.max() - img_array.min())#
        
        unique_values=np.unique(img_array)
        
        # Generate a random mask with the same shape as img_array
        mask = np.random.rand(*img_array.shape) < self.change_prob
        
        # Apply the mask to randomly change the pixels to any of the unique values
        random_values = np.random.choice(unique_values, size=img_array.shape)
        img_array[mask] = random_values[mask]
        
        return img_array.transpose(1, 2, 0)



In [13]:
Dataset="Gutenberg"
(train_x, train_y), (valid_x, valid_y), (test_x), metadata = load_datasets(Dataset, truncate=False)
test_y = np.load(os.path.join('../../datasets_val/'+Dataset,'test_y.npy'))
select_augment="None"
data_processor = DataProcessor(train_x[:], train_y[:], valid_x, valid_y, test_x, metadata, select_augment)
train_loader, valid_loader, test_loader = data_processor.process()


metadata
dataset=metadata["codename"]

ic| self.select_augment: 'None'
ic| f"selected transform {train_transform}": 'selected transform []'
ic| self.x.shape: torch.Size([45000, 1, 27, 18])
ic| self.x.shape: torch.Size([15000, 1, 27, 18])
ic| self.x.shape: torch.Size([6000, 1, 27, 18])


In [14]:
rg=RegNet(metadata,
                    W0=[16, 120, 8],
                    WA=[16, 64, 8],
                    WM=[2.05,2.9,0.05],
                    D=[8,22,1], 
                    G=[8,8,8], 
                    base_config=f"../configs/search_space/config.yaml")

folder=f"/home/woody/iwb3/iwb3021h/NAS_COMPETITION_RESULTS/stages_pool"


In [15]:
unique_values=np.unique(valid_loader.dataset.x[0])
C,H,W=metadata['input_shape'][1:4]
PH,PW=int(H/8),int(W/8)
poss_augs= [

        [],
        [v2.RandAugment(magnitude=9)],
        [v2.RandAugment(magnitude=5)],
        [v2.RandAugment(magnitude=1)],
        [v2.TrivialAugmentWide(num_magnitude_bins=31)],
        [v2.TrivialAugmentWide(num_magnitude_bins=15)],
        [v2.AugMix(severity=3)],
        [v2.AugMix(severity=1)],
        ##########################
        [v2.RandomHorizontalFlip(),v2.RandomVerticalFlip()],
        [v2.RandomErasing(p=0.2, scale=(0.05, 0.2), ratio=(0.3, 3.3)), v2.RandomHorizontalFlip(),v2.RandomVerticalFlip()],
        [v2.RandomErasing(p=0.2, scale=(0.05, 0.2), ratio=(0.3, 3.3))],
        [v2.RandomErasing(p=0.2, scale=(0.02, 0.2), ratio=(0.3, 3.3)), v2.RandomCrop((H,W), padding=(PH,PW))],
        [v2.RandomCrop((H,W), padding=(PH,PW))],
        [v2.RandomCrop((H,W), padding=(PH,PW)), v2.RandomHorizontalFlip(),v2.RandomVerticalFlip()],
        [v2.RandomErasing(p=0.2, scale=(0.02, 0.2), ratio=(0.3, 3.3)),v2.RandomCrop((H,W), padding=(PH,PW)),v2.RandomHorizontalFlip()],
        ###########################################################
        [RandomPixelChange(0.01), v2.ToTensor()],
        [RandomPixelChange(0.025), v2.ToTensor()],
        [RandomPixelChange(0.05), v2.ToTensor()],
        [RandomPixelChange(0.01), v2.ToTensor(), v2.RandomHorizontalFlip(),v2.RandomVerticalFlip()],
        [RandomPixelChange(0.01), v2.ToTensor(),v2.RandomErasing(p=0.2, scale=(0.05, 0.2), ratio=(0.3, 3.3))],
        [RandomPixelChange(0.01), v2.ToTensor(), v2.RandomCrop((H,W), padding=(PH,PW))],
        [RandomPixelChange(0.01), v2.ToTensor(),v2.RandomHorizontalFlip(),v2.RandomVerticalFlip(), v2.RandomErasing(p=0.2, scale=(0.05, 0.2), ratio=(0.3, 3.3))]

    ]



/apps/jupyterhub/jh3.1.1-py3.11/envs/pytorch-2.2.0/lib/python3.12/site-packages/torchvision/transforms/v2/_deprecated.py:41: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.
  warnings.warn(


In [16]:
train_data=get_predictor_data_from_folder(base_folder=folder,
                                          studies=[dataset])

['provocative_skua', 'passionate_hornet', 'rousing_cuckoo', 'modest_snail', 'glossy_fennec', 'versed_duck', 'giga_jackrabbit', 'amigurumi_camel', 'splendid_duck', 'amiable_smilodon', 'sweet_cat', 'handsome_eel', 'roaring_chameleon', 'meticulous_oyster', 'cunning_bandicoot', 'invaluable_leopard', 'organic_limpet', 'voracious_kagu', 'macho_seriema', 'clay_honeybee', 'jolly_spoonbill', 'loose_python', 'peach_marmot', 'graceful_flamingo', 'enthusiastic_binturong', 'xanthic_serval', 'spiritual_raptor', 'beige_limpet', 'dark_tuatara', 'therapeutic_frigatebird', 'prophetic_grouse', 'solid_chicken', 'tidy_silkworm', 'strong_finch', 'perfect_waxbill', 'crystal_walrus', 'pragmatic_roadrunner', 'aspiring_toad', 'muscular_poodle', 'fanatic_cheetah', 'nano_ibis', 'peridot_starling', 'maize_bulldog', 'stirring_seal', 'cherry_tapir', 'olivine_goat', 'fascinating_tarsier', 'married_bonobo', 'magic_deer', 'neat_termite', 'peridot_baboon', 'nice_leech', 'modest_hound', 'muscular_bullmastiff', 'refined_t

In [ ]:
train_data

In [17]:
individuals=os.listdir(folder+"/"+dataset)
individuals=[ind for ind in individuals if os.path.isdir(os.path.join(folder+"/"+dataset, ind)) and ".ipynb" not in ind]
individuals = random.sample(individuals, 10)

In [18]:
preload_models=False
if preload_models:
    models={}
    for individual in individuals:
            ind_config=f"{folder}/{dataset}/{individual}/config.yaml"
            models[individual], _=rg.load_model(config_file=ind_config)
#models, chromosomes=rg.load_generation(folder+"/"+dataset)

In [19]:
from torch.utils.data import DataLoader, TensorDataset
from icecream import ic
import gc
##########################
train_loader.shuffle=False
tot_dfs=[]
for aug in range(len(poss_augs)):
    current_transforms=train_loader.dataset.transform.transforms
    train_loader.dataset.transform=v2.Compose(poss_augs[aug]+[current_transforms[-1]])
    ic("#############")
    ic(aug)
    ic(train_loader.dataset.transform.transforms)
    ###############
   
    #if "RandAugment"
    train_loader_iter = iter(train_loader)
    # Number of batches to extract
    num_batches =5

    # Initialize lists to hold the inputs and targets from the first 5 batches
    for batch in range(num_batches):
        inputs_list = []
        targets_list = []
        try:
            #for _ in range(num_batches):
            inputs, targets = next(train_loader_iter)  # Get the next batch
            inputs_list.append(inputs)
            targets_list.append(targets)
        except:
            continue
        # Concatenate the inputs and targets across the batches
        inputs = torch.cat(inputs_list)
        targets = torch.cat(targets_list)

        # Create a new TensorDataset from the selected data
        new_dataset = TensorDataset(inputs, targets)

        # Create a new DataLoader from this new dataset
        new_valid_loader = DataLoader(new_dataset, batch_size=train_loader.batch_size, shuffle=False)
        #train_loader.batch_size

        syn_scores={}
        measures=["fisher", "jacob_cov","plain"]

        for individual in individuals:
                        ind_config=f"{folder}/{dataset}/{individual}/config.yaml"
                        model_aux, _=rg.load_model(config_file=ind_config)
                        #model_aux, _=regnet_space.create_model(params=params_dict[individual],save_folder=None, name=individual, gen=None, config_updates=None)
                        ic(len(new_valid_loader))
                        syn_scores[individual]=find_measures(model_aux.to("cuda"), new_valid_loader, ("random",len(new_valid_loader),metadata["num_classes"]), "cuda", F.cross_entropy, measures )

                        #syn_scores[individual]["params"]=str(params_dict[individuals])
                        del model_aux
                        gc.collect

    #########################################
        tot_df=pd.DataFrame(syn_scores).T.reset_index().rename(columns={"index":"name"})
        tot_df["aug"]=aug
        tot_df["batch"]=batch
        tot_dfs.append(tot_df)
    #tot_df.to_csv(f"zcost_proxies/results/augs/{dataset}_t{aug}.csv")
tot_dfs=pd.concat(tot_dfs)


    #for metric in ["grasp","fisher","grad_norm","jacob_cov","plain","snip","synflow"]:
    #    if metric=="jacob_cov":
    #        df_syn[f"rank_{metric}"]=df_syn[metric].rank(ascending=False).astype(int)
    #    else:
    #        df_syn[f"rank_{metric}"]=df_syn[metric].rank().astype(int)

    #from scipy.stats import spearmanr
    #dict_spearman={}
    #dict_pearson={}
    #for metric in ["grasp","fisher","grad_norm","jacob_cov","plain","snip","synflow"]:
    ##for metric in ["snip"]:
    #        spearman_corr, _ = spearmanr(df_syn['best_acc'],df_syn[f"{metric}"])
    #        pearson_corr= df_syn['best_acc'].corr(df_syn[metric])

    #        dict_spearman[metric]=spearman_corr
    #        dict_pearson[metric]=pearson_corr

    #corr_df=pd.DataFrame([dict_spearman])
    #corr_df["dataset"]=dataset
    #df_syn.to_csv(f"zcost_proxies/results/augs/{dataset}_t{aug}.csv")

    #####################################################

ic| '#############'
ic| aug: 0
ic| train_loader.dataset.transform.transforms: [Normalize(mean=tensor([0.0370]), std=tensor([0.1889]))]
ic| len(new_valid_loader): 1


Loading model: /home/woody/iwb3/iwb3021h/NAS_COMPETITION_RESULTS/stages_pool/Gutenberg/voracious_kagu/config.yaml


/apps/jupyterhub/jh3.1.1-py3.11/envs/pytorch-2.2.0/lib/python3.12/site-packages/torch/nn/modules/module.py:1352: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
ic| len(new_valid_loader): 1


Loading model: /home/woody/iwb3/iwb3021h/NAS_COMPETITION_RESULTS/stages_pool/Gutenberg/golden_scorpion/config.yaml


ic| len(new_valid_loader): 1


...

ic| len(new_valid_loader): 1


Loading model: /home/woody/iwb3/iwb3021h/NAS_COMPETITION_RESULTS/stages_pool/Gutenberg/prudent_crab/config.yaml


ic| len(new_valid_loader): 1


Loading model: /home/woody/iwb3/iwb3021h/NAS_COMPETITION_RESULTS/stages_pool/Gutenberg/delectable_octopus/config.yaml


KeyboardInterrupt: 

In [ ]:
from scipy.stats import pearsonr
pearsonr(tot_dfs_iter.groupby(["name","aug"]).mean()["fisher"].reset_index().sort_values(by=["name","aug"])["fisher"].values, tot_dfs.sort_values(by=["name","aug"])["fisher"].values)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
individuals=tot_dfs.name.unique()
num_batches=tot_dfs.batch.max()+1
rank_batches_fisher=[]
for individual in individuals:
    for batch in range(num_batches):
        subset_df = tot_dfs[(tot_dfs.name == individual) & (tot_dfs.batch == batch)].sort_values(by="aug").set_index("aug")
        Q1 = subset_df["fisher"].quantile(0.1)
        Q3 = subset_df["fisher"].quantile(0.9)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        subset_df_no_outliers = subset_df[(subset_df["fisher"] >= lower_bound) & (subset_df["fisher"] <= upper_bound)]
        scaler = MinMaxScaler()
        subset_df_no_outliers["fisher"] = scaler.fit_transform(subset_df_no_outliers[["fisher"]])
        rank_batches_fisher.append(subset_df_no_outliers["fisher"])    

rank_batches_fisher=pd.concat(rank_batches_fisher)
rank_batches_fisher=rank_batches_fisher.reset_index().groupby("aug").mean().sort_values(by="fisher")

rank_batches_jac=[]
for individual in individuals:
    for batch in range(num_batches):
        subset_df = tot_dfs[(tot_dfs.name == individual) & (tot_dfs.batch == batch)].sort_values(by="aug").set_index("aug")
        Q1 = subset_df["jacob_cov"].quantile(0.1)
        Q3 = subset_df["jacob_cov"].quantile(0.9)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        subset_df_no_outliers = subset_df[(subset_df["jacob_cov"] >= lower_bound) & (subset_df["jacob_cov"] <= upper_bound)]
        scaler = MinMaxScaler()
        subset_df_no_outliers["jacob_cov"] = scaler.fit_transform(subset_df_no_outliers[["jacob_cov"]])
        rank_batches_jac.append(subset_df_no_outliers["jacob_cov"])      

rank_batches_jac=pd.concat(rank_batches_jac)
rank_batches_jac=rank_batches_jac.reset_index().groupby("aug").mean().sort_values(by="jacob_cov")


In [ ]:
tot_dfs=pd.read_csv("zcost_proxies/results/augs/ranks_LaMelo.csv",index_col=0)

In [ ]:

for col in ["fisher","jacob_cov"]:
    try:
        tot_dfs[col]=tot_dfs[col].astype(float)
    except:
        tot_dfs[col]=0

In [ ]:
#rank_batches=pd.concat([rank_batches_fisher,rank_batches_jac,rank_batches_plain,rank_batches_syn, rank_batches_snip,rank_batches_grad],axis=1)
rank_batches=pd.concat([rank_batches_fisher,rank_batches_jac], axis=1)
#rank_batches=pd.concat([rank_batches_fisher], axis=1)
rank_batches["total_rank"]=rank_batches["fisher"]+rank_batches["jacob_cov"]#-rank_batches["plain"]#-rank_batches["synflow"]
#rank_batches["total_rank"]=rank_batches["fisher"]+rank_batches["jacob_cov"]-rank_batches["snip"]#-rank_batches["grad_norm"]+rank_batches["plain"]
#rank_batches["total_rank"]=rank_batches["jacob_cov"]#-rank_batches["snip"]-rank_batches["grad_norm"]




In [ ]:
import pandas as pd
test_scores_df=[]
for i in range(23):
    try:
        #df_aux=pd.read_csv(f"/home/woody/iwb3/iwb3021h/THESIS_RESULTS/full_training_evonas/finetuning/{dataset}/augmentations_test_1/{dataset}/aug_{i}/test_acc.csv", index_col=0)
        #df_aux["aug"]=i
        #test_scores_df.append(df_aux)
        
        #df_aux["model"]="regnet"
        
        #df_aux2=pd.read_csv(f"/home/woody/iwb3/iwb3021h/THESIS_RESULTS/full_training_evonas/finetuning/{dataset}/augmentations_test_2/{dataset}/aug_{i}/test_acc.csv", index_col=0)
        #test_scores_df.append(df_aux2)
        #df_aux2["aug"]=i
        
        df_aux3=pd.read_csv(f"/home/woody/iwb3/iwb3021h/THESIS_RESULTS/full_training_evonas/finetuning/{dataset}/augmentations_test_resnet/{dataset}/aug_{i}/test_acc.csv", index_col=0)
        test_scores_df.append(df_aux3)
        df_aux3["aug"]=i
        #df_aux3["model"]="resnet"

    except:
        print(f"{i} not possible")
    
    
test_scores_df=pd.concat(test_scores_df)
test_scores_df=test_scores_df.groupby("aug").mean().reset_index()

#vals=pd.read_json(f"/home/woody/iwb3/iwb3021h/THESIS_RESULTS/full_training_evonas/finetuning/{dataset}/augmentations_test_1/{dataset}/augmentation_results.json").T.reset_index().rename(columns={"index":"aug"})
#vals2=pd.read_json(f"/home/woody/iwb3/iwb3021h/THESIS_RESULTS/full_training_evonas/finetuning/{dataset}/augmentations_test_2/{dataset}/augmentation_results.json").T.reset_index().rename(columns={"index":"aug"})
vals3=pd.read_json(f"/home/woody/iwb3/iwb3021h/THESIS_RESULTS/full_training_evonas/finetuning/{dataset}/augmentations_test_resnet/{dataset}/augmentation_results.json").T.reset_index().rename(columns={"index":"aug"})
vals=pd.concat([vals3])
#vals=vals.groupby("aug").mean().reset_index()